<a href="https://colab.research.google.com/github/nagasivaninandam/Chatbot-LLM-Frontend-using-Streamlit/blob/master/Chatbot_LLM_Frontend_using_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install streamlit==1.36.0
# download a small static cloudflared binary (no login/token needed)
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.16.0 requires watchdog<7.0.0,>=6.0.0, but you have watchdog 4.0.2 which is incompatible.


In [6]:
%%writefile app.py
import re
import random
import time
from datetime import datetime
import streamlit as st

# ---------- Page config ----------
st.set_page_config(
    page_title="Lite Chatbot • Streamlit",
    page_icon="💬",
    layout="centered",
    initial_sidebar_state="expanded"
)

# ---------- Sidebar ----------
with st.sidebar:
    st.markdown("## ⚙️ Settings")
    ui_theme = st.selectbox(
        "UI Theme",
        ["Light (recommended)", "Dark"],
        help="Switch UI theme without restarting."
    )
    mode = st.selectbox(
        "Response style",
        ["Helpful", "Echo", "Eliza-ish", "Concise"],
        help="Pick how the local toy responder should behave."
    )
    temperature = st.slider(
        "Creativity (temperature)", 0.0, 1.0, 0.25, 0.05,
        help="Adds a bit of randomness to the toy responses."
    )
    typing_speed = st.slider(
        "Typing effect speed (ms/char)", 0, 30, 7, 1,
        help="Visual typing effect for replies (0 disables it)."
    )

    st.markdown("---")
    if st.button("🧹 Clear chat history"):
        st.session_state.pop("messages", None)
        st.rerun()
    st.markdown("### ℹ️ About")
    st.caption(
        "Pure Streamlit demo chat UI (no external APIs). "
        "Nice for prototyping frontends entirely in Colab."
    )

# ---------- Themes (CSS) ----------
LIGHT_CSS = """
<style>
:root {
  --bg: #ffffff;
  --text: #0f172a;
  --muted: #475569;
  --card: #ffffff;
  --card-border: rgba(2, 6, 23, 0.06);
  --shadow: 0 8px 24px rgba(2, 6, 23, 0.08);
  --user: #f3f4f6;
  --bot: #e0f2fe;
  --bot-text: #0c4a6e;
  --accent: #2563eb;
  --radius: 16px;
}
html, body, .main { background: var(--bg) !important; color: var(--text) !important; }

.container {
  max-width: 900px;
  margin: 0 auto;
}

.chat-card {
  background: var(--card);
  border: 1px solid var(--card-border);
  border-radius: 20px;
  padding: 18px;
  box-shadow: var(--shadow);
}

h1, h2, h3, .stMarkdown p { color: var(--text) !important; }

.user-bubble, .bot-bubble {
  padding: 12px 14px;
  border-radius: var(--radius);
  margin: 8px 0;
  width: fit-content;
  max-width: 100%;
  line-height: 1.55;
  word-wrap: break-word;
  white-space: pre-wrap;
  border: 1px solid var(--card-border);
}

.user-bubble {
  background: var(--user);
}

.bot-bubble {
  background: var(--bot);
  color: var(--bot-text);
  font-weight: 600;
}

.footer {
  color: var(--muted);
  font-size: 13px;
}
</style>
"""

DARK_CSS = """
<style>
:root {
  --bg: #0e1117;
  --text: #e5e7eb;
  --muted: #9ca3af;
  --card: rgba(255,255,255,0.03);
  --card-border: rgba(255,255,255,0.08);
  --shadow: 0 10px 25px rgba(0,0,0,0.25);
  --user: #1f2937;
  --bot: #0ea5e9;
  --bot-text: #082f49;
  --accent: #22d3ee;
  --radius: 16px;
}
html, body, .main { background: var(--bg) !important; color: var(--text) !important; }

.container { max-width: 900px; margin: 0 auto; }

.chat-card {
  background: var(--card);
  border: 1px solid var(--card-border);
  border-radius: 20px;
  padding: 18px;
  box-shadow: var(--shadow);
}

.user-bubble, .bot-bubble {
  padding: 12px 14px;
  border-radius: var(--radius);
  margin: 8px 0;
  width: fit-content;
  max-width: 100%;
  line-height: 1.55;
  word-wrap: break-word;
  white-space: pre-wrap;
  border: 1px solid var(--card-border);
}

.user-bubble { background: linear-gradient(135deg, #1f2937 0%, #111827 100%); }
.bot-bubble { background: linear-gradient(135deg, #0ea5e9 0%, #22d3ee 100%); color: var(--bot-text); font-weight: 600; }

.footer { color: var(--muted); font-size: 13px; }
</style>
"""

st.markdown(LIGHT_CSS if "Light" in ui_theme else DARK_CSS, unsafe_allow_html=True)

# ---------- Session State ----------
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Hey! I’m an offline demo bot. Ask me anything 🤖"}
    ]

# ---------- Tiny local 'toy model' ----------
def toy_llm(prompt: str, mode: str = "Helpful", temperature: float = 0.2) -> str:
    prompt_clean = prompt.strip()
    lower = prompt_clean.lower()

    helpers = [
        "Here’s a quick thought:",
        "A helpful hint:",
        "One simple approach:",
        "Try this:",
        "You could consider:"
    ]
    emojis = ["💡", "✨", "✅", "📌", "🧠", "🔧"]

    # keyword reactions
    if re.search(r"\b(hello|hi|hey)\b", lower):
        return random.choice([
            "Hey there 👋 What’s on your mind?",
            "Hello! How can I help today?",
            "Hi! Fire away with a question."
        ])
    if "time" in lower:
        return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}."
    if "joke" in lower:
        return random.choice([
            "Why did the developer go broke? Because they used up all their cache. 😅",
            "I told my computer I needed a break, and it said: ‘No problem, I’ll go to sleep.’ 💤",
            "There are 10 types of people in the world: those who understand binary and those who don’t. 😜"
        ])

    if mode == "Echo":
        return f"You said: “{prompt_clean}”"
    if mode == "Eliza-ish":
        reflections = [
            ("i am", "you are"), ("i'm", "you are"), ("i", "you"),
            ("my", "your"), ("me", "you"), ("am", "are")
        ]
        resp = prompt_clean
        for a, b in reflections:
            resp = re.sub(rf"\b{a}\b", b, resp, flags=re.IGNORECASE)
        return f"How long have you felt that {resp}?"
    if mode == "Concise":
        base = ["Okay.", "Done.", "Sure.", "Noted.", "Got it.", "In short:"]
        return random.choice(base) + " " + " ".join(prompt_clean.split()[:16]).rstrip(".") + "."

    tips = [
        "break your task into smaller steps",
        "write down assumptions first",
        "start with a simple version then iterate",
        "add print/log statements to observe behavior",
        "verify inputs and outputs with small tests"
    ]
    n_extra = 1 + int(temperature * 4)
    chosen_tips = random.sample(tips, min(len(tips), n_extra))
    return f"{random.choice(helpers)} {random.choice(emojis)} You could {', then '.join(chosen_tips)}."

# ---------- Header ----------
st.markdown('<div class="container">', unsafe_allow_html=True)
st.markdown("# 💬 Lite Chatbot")
st.caption("Attractive Streamlit UI • No external APIs • Works in Colab")

# ---------- Chat History ----------
st.markdown('<div class="chat-card">', unsafe_allow_html=True)
for msg in st.session_state.messages:
    if msg["role"] == "user":
        st.markdown(f'<div class="user-bubble">👤 {msg["content"]}</div>', unsafe_allow_html=True)
    else:
        st.markdown(f'<div class="bot-bubble">🤖 {msg["content"]}</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# ---------- Input ----------
user_input = st.chat_input("Type a message…")

if user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})
    reply = toy_llm(user_input, mode=mode, temperature=temperature)
    if typing_speed <= 0:
        st.session_state.messages.append({"role": "assistant", "content": reply})
    else:
        typed = ""
        placeholder = st.empty()
        for ch in reply:
            typed += ch
            placeholder.markdown(f'<div class="bot-bubble">🤖 {typed}</div>', unsafe_allow_html=True)
            time.sleep(typing_speed / 1000.0)
        placeholder.empty()
        st.session_state.messages.append({"role": "assistant", "content": reply})
    st.rerun()

# ---------- Footer ----------
st.markdown("—")
st.markdown('<div class="footer">Demo only • No data leaves this notebook • Built with ❤️ and Streamlit</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)


Overwriting app.py


In [3]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 >/dev/null 2>&1 &


In [7]:
# Start a quick tunnel to the Streamlit server; log to a file so we can parse the URL
!./cloudflared tunnel --url http://localhost:8501 --logfile cloudflared.log --loglevel info >/dev/null 2>&1 &


In [5]:
import re, time, pathlib
log = pathlib.Path("cloudflared.log")
# wait up to ~30s for the URL to appear
for _ in range(30):
    if log.exists():
        txt = log.read_text()
        m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", txt)
        if m:
            print("Your public app URL:\n", m.group(0))
            break
    time.sleep(1)
else:
    print("Could not detect the tunnel URL yet. Re-run this cell in a few seconds to refresh.")


Your public app URL:
 https://racing-liberty-effect-transmitted.trycloudflare.com
